### The dataset contains two columns first contains the tweets and the second contains four class (figurative, irony, sarcasm, regular ) we have to predict the class of the tweet.

In [3]:
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import numpy as np
import pandas as pd
import re
import time 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.models import Sequential #(used to initialise the nural network)
from keras.layers import Dense #(used to create the layers in ANN)
from tensorflow.keras import layers

In [4]:
data = pd.read_csv('train.csv')

In [5]:
data.head(5)

,tweets,class
0,Be aware dirty step to get money #staylight ...,figurative
1,#sarcasm for #people who don't understand #diy...,figurative
2,@IminworkJeremy @medsingle #DailyMail readers ...,figurative
3,@wilw Why do I get the feeling you like games?...,figurative
4,-@TeacherArthurG @rweingarten You probably jus...,figurative


In [6]:
data.isnull().sum()

tweets    0
class     0
dtype: int64

In [7]:
data.shape

(81408, 2)

In [8]:
data['class'].value_counts()

figurative    21238
irony         20894
sarcasm       20681
regular       18595
Name: class, dtype: int64

### Cleaning the Tweets 

In [8]:
# Cleaning the texts
corpus = []
for i in range(0, 81408):
    tweet = re.sub('[^a-zA-Z]', ' ', data['tweets'][i])
    tweet = tweet.lower()
    tweet = tweet.split()
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

In [10]:
#save the corpus
np.save('tweet_cleaned_corpus.npy', corpus)

In [9]:
#load corpus
corpus = np.load('tweet_cleaned_corpus.npy', mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')

### TF/IDF vectorizer 

##### Creating TF/IDF matrix to calculate the term weight of the words.  
max_df = 0.50 means "It ignores terms that appear in more than 50% of the documents".min_df = 0.01 means "ignore terms that appear in less than 1% of the documents".
min_df = 5 means "ignore terms that appear in less than 5 documents".

In [10]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, stop_words = {'english'},ngram_range=(1, 2),min_df=5, max_df = 0.3)
 
X_tf = tfidf_vectorizer.fit_transform(corpus)
y = data.iloc[:, 1].values

### As we have 4 class categories so we are going to encode the output classes

In [11]:
classes = list(set(y))

In [12]:
Y_encoded = []
for data_point in y:
    data_point_encoded = [0] * len(classes)
    for i in range(len(classes)):
        if classes[i] == data_point:
            data_point_encoded[i] = 1
    Y_encoded.append(data_point_encoded)

### Splitting the dataset into the Training set and Test set

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_tf.toarray(), Y_encoded, test_size = 0.25, random_state = 0, stratify = y)

### Model Creation 

In [14]:
model = Sequential()
model.add(Dense(units=512, init = 'uniform', activation='sigmoid',input_dim=len(X_train[0])))
# model.add(Dropout(0.2))
# model.add(Dense(units=409, activation='sigmoid'))
# model.add(Dropout(0.2))
model.add(Dense(units=4, activation='softmax'))

We have created the model with input nodes equal to len(X_train[0])
init = uniform will assign random weights to the layers initially.
we have used activation function sigmoid in hidden layers as it will give continous output to next layer
in output layer we have used softmax activation function as the output is multiclass classification. 

In [15]:
model.compile(loss=categorical_crossentropy,optimizer=SGD(lr=0.01,momentum=0.9, nesterov=True), metrics=['accuracy'])

We have used SGD optimiser as the chances of getting local minima is very high, so taking the step by step approcah to get the minima with SGD.

### while compiling we are using "categorical_crossentropy" as the loss function as the output is multiclass(4) 

In [18]:
#Fitting the ANN to the training set 
history = model.fit(np.array(X_train), np.array(y_train), epochs=75, batch_size=64, verbose=1, validation_split=0.1)


Train on 54950 samples, validate on 6106 samples
Epoch 1/75
54950/54950 [==============================] - 112s 2ms/step - loss: 1.3985 - accuracy: 0.2562 - val_loss: 1.3908 - val_accuracy: 0.2563
Epoch 2/75
54950/54950 [==============================] - 112s 2ms/step - loss: 1.3845 - accuracy: 0.2768 - val_loss: 1.3896 - val_accuracy: 0.4309
Epoch 3/75
54950/54950 [==============================] - 111s 2ms/step - loss: 1.3599 - accuracy: 0.3163 - val_loss: 1.3425 - val_accuracy: 0.2622
Epoch 4/75
54950/54950 [==============================] - 114s 2ms/step - loss: 1.3064 - accuracy: 0.4016 - val_loss: 1.2627 - val_accuracy: 0.7270
Epoch 5/75
54950/54950 [==============================] - 113s 2ms/step - loss: 1.2157 - accuracy: 0.5224 - val_loss: 1.1611 - val_accuracy: 0.4720
Epoch 6/75
54950/54950 [==============================] - 111s 2ms/step - loss: 1.0903 - accuracy: 0.6118 - val_loss: 1.0359 - val_accuracy: 0.6105
Epoch 7/75
54950/54950 [==============================] - 113s

#### when we fit the model with the training set we are giving validation split =0.1 i.e. the model will get validated in 10% of the data, the batch size is 64 so after every 64 rows the weight of the neural network is getting updated, the epochs can be changed to run the whole dataset again so that it can decrease the loss and increase the accuracy.   


In [19]:
model.model.save("tweet_predictor.h5")

In [20]:
predictions = [np.argmax(pred) for pred in model.predict(np.array(X_test))]

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               12939264  
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2052      
Total params: 12,941,316
Trainable params: 12,941,316
Non-trainable params: 0
_________________________________________________________________


In [25]:
correct = 0
for i in range(len(predictions)):
    if predictions[i] == np.argmax(y_test[i]):
        correct += 1
print ("Correct:", correct)
print ("Total:", len(predictions))

Correct: 14049
Total: 20352
